In [3]:
# import faiss
import numpy as np
import dspy
from dspy.retrieve.faiss_rm import FaissRM
from dspy.embedding import SentenceTransformers # Make sure this matches your embedding model!

# --- Configuration ---
FAISS_INDEX_PATH = "path/to/your/ideas1.faiss" # Ensure this path is correct

# THIS IS YOUR MY_CORPUS_PASSAGES LIST, EXACTLY AS YOU PROVIDED IT:
MY_CORPUS_PASSAGES = [
    "1) Maximize the Beauty - fully channel the beauty with in. Maybe ask what makes this moment beautiful? See if beauty can be increased in every situation. MtB also could be taken as a use of reason and also a disciplining of the senses to focus on beauty (i.e. all the pretty flowers, all the pretty birds). (Update 3-16-21 -This is one of the first 4 points created because they were on the top of my mind when I first started this project. Maximize the Beauty and Full Expression still are some of the concepts I am most influenced by, create an ethos around, consider to be primary to my concept of virtue, and aim to implement as much as possible.) ",
    "2) Full Expression - it takes a lot of effort for one to understand who they are when they are comfortable and how to channel the most real expressions of themselves what holds people back? Shyness, distraction (inability to focus on that which they want express)",
    "3) Expect Rising - this means our expectations are constantly rising. Kind of in line with \"give em an inch they'll take a mile\" also related to law of diminishing returns -\"The law of diminishing returns states that in all productive processes, adding more of one factor of production, while holding all others constant (\"ceteris paribus\"), will at some point yield lower incremental per-unit returns.\" Tooo much ice cream too much cash",
    "4) The Power of Pettiness - is the idea that pettiness is the destroyer of all people. That even the best of us can’t truly be unaffected by the petty bullshit around us. They can however minimize its effects. In my own life I am obliterated by friends and coworker’s snide remarks and judgments. How do I minimize its effects? By talking myself down, deep breaths, weed, alcohol, revenge...(this is the end of the first four points that were the strongest when I first started making these ideas on a small scrap of paper from the sides on union film sets.)",
    "5) Various meditation - I am a big fan of meditation techniques that help train the mind from moment to moment. There are various techniques that help provide concentration and awareness in the passing moments within shorter intervals than traditional formal meditation. While using a timer to make sure you meditate for at least 10 minutes is helpful. A stopwatch to monitor your awareness of time and to also use time to monitor your concentration and awareness. A and C are essential to have Full Expression. Update 2-26-19 - Passive observation ala Krishnamurti. Just chill. Dont do anything. Just observe whatever is present. Don't react to thoughts just observe them. Don't interact with them. If they are mind blowing write them down. Then return to just chilling and observing."
]


# --- The rest of your setup is as previously described ---

# 1. Load the FAISS index
try:
    faiss_index = faiss.read_index(FAISS_INDEX_PATH)
    print(f"FAISS index loaded from {FAISS_INDEX_PATH}.")
    print(f"Number of vectors in index: {faiss_index.ntotal}")
    print(f"Vector dimension: {faiss_index.d}")

    if faiss_index.ntotal != len(MY_CORPUS_PASSAGES):
        raise ValueError(
            f"Mismatch: FAISS index has {faiss_index.ntotal} vectors, "
            f"but MY_CORPUS_PASSAGES has {len(MY_CORPUS_PASSAGES)} entries. "
            "These must match precisely."
        )

except Exception as e:
    print(f"Error loading FAISS index '{FAISS_INDEX_PATH}': {e}")
    print("Ensure the file exists and is valid. Also, check MY_CORPUS_PASSAGES.")
    exit()

# 2. Initialize your Embedding Model (Crucial: Must match what was used for embedding!)
embedding_model = SentenceTransformers('sentence-transformers/all-MiniLM-L6-v2')
# If you used a different one (e.g., from OpenAI, Cohere, etc.), change this line.

# 3. Configure DSPy to Use Your FAISS Retriever
retriever_module = FaissRM(index=faiss_index, embedding_model=embedding_model, passages=MY_CORPUS_PASSAGES)
dspy.settings.rm = retriever_module

# Configure your main Language Model (LLM) for DSPy
dspy.settings.lm = dspy.OllamaLocal(model='llama3') # Example: using a local Ollama model

print("DSPy configured with FaissRM using 'ideas1.faiss' and your embedding model.")


# 4. Integrate dspy.Retrieve into Your DSPy Modules
class MyIdeasRAGProgram(dspy.Module):
    def __init__(self, num_passages=2): # You might want 1 or 2 passages for these short ideas
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        # Use ChainOfThought to encourage the LLM to think about the concepts
        self.generate_response = dspy.ChainOfThought("context, question -> answer")

    def forward(self, question):
        context = self.retrieve(question).passages
        # Adapt the prompt for conceptual ideas
        response = self.generate_response(
            context=context,
            question=question
        )
        return response

# --- Example Usage ---
rag_system = MyIdeasRAGProgram(num_passages=1) # Let's try retrieving just the top relevant idea first

# Example questions based on your ideas:
questions = [
    "How can one maximize beauty?",
    "What prevents full self-expression?",
    "Explain the concept of 'expect rising'.",
    "What is 'The Power of Pettiness' about?",
    "Tell me about different meditation techniques mentioned."
]

for q in questions:
    print(f"\n--- Question: {q} ---")
    response = rag_system(question=q)
    print(f"Retrieved Context (first passage):")
    if response.context:
        print(f"  '{response.context[0]}'")
    else:
        print("  No context retrieved.")
    print(f"Generated Answer: {response.answer}")
    print("-" * 30)

ImportError: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.